# Loading Data Sets

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
def scan_directory(directory_path, file_extension=".xlsx"):
    data_files = []
    try:
        directory = Path(directory_path)
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(file_extension):
                    file_path = Path(root) / file
                    relative_path = str(file_path.relative_to(directory))
                    data_files.append(relative_path)

        return data_files
    except Exception as e:
        print("❌ Error scanning directory: {e}")
        return []


In [3]:
def read_files(files_list):
    dfs = []
    for file in files_list:
        df = pd.read_excel(file)
        dfs.append(df)

    return dfs

# Data Cleanup and Feature Engineering

In [4]:
def create_clean_df(list_of_dfs):
    merged_df = list_of_dfs[0]
    for i in range(1, len(list_of_dfs)):
        merged_df = pd.merge(merged_df, list_of_dfs[i], on="Name", how="inner")

    merged_df = merged_df.dropna(axis=1, how="all")

    return merged_df

In [5]:
# Kyte-Doolitle (GRAVY) table
KD = {
    "A": 1.8,
    "R": -4.5,
    "N": -3.5,
    "D": -3.5,
    "C": 2.5,
    "Q": -3.5,
    "E": -3.5,
    "G": -0.4,
    "H": -3.2,
    "I": 4.5,
    "L": 3.8,
    "K": -3.9,
    "M": 1.9,
    "F": 2.8,
    "P": -1.6,
    "S": -0.8,
    "T": -0.7,
    "W": -0.9,
    "Y": -1.3,
    "V": 4.2,
}

HYDROPHOBIC = set("AILMVFWYV")
AROMATIC = set("FWY")
POSITIVE = set("KRH")
NEGATIVE = set("DE")
POLAR = set("STNQ")

In [6]:
def aa_counts(seq):
    counts = {aa: 0 for aa in KD.keys()}
    if isinstance(seq, str):
        for ch in seq.upper():
            if ch in counts:
                counts[ch] += 1

    return counts

In [7]:
def seq_features(seq, prefix):
    counts = aa_counts(seq)
    L = sum(counts.values())
    feats = {f"{prefix}_len": float(L)}

    if L == 0:
        for k in [
            "hydrophobic_frac",
            "aromatic_frac",
            "positive_frac",
            "negative_frac",
            "polar_frac",
            "kd_gravy",
        ]:
            feats[f"{prefix}_{k}"] = 0.0
        for aa in KD.keys():
            feats[f"{prefix}_comp_{aa}"] = 0.0

        return feats

    feats[f"{prefix}_hydrophobic_frac"] = (
        sum(counts[a] for a in HYDROPHOBIC if a in counts) / L
    )
    feats[f"{prefix}_aromatic_frac"] = (
        sum(counts[a] for a in AROMATIC if a in counts) / L
    )
    feats[f"{prefix}_positive_frac"] = (
        sum(counts[a] for a in POSITIVE if a in counts) / L
    )
    feats[f"{prefix}_negative_frac"] = (
        sum(counts[a] for a in NEGATIVE if a in counts) / L
    )
    feats[f"{prefix}_polar_frac"] = sum(counts[a] for a in POLAR if a in counts) / L
    feats[f"{prefix}_kd_gravy"] = sum(KD[a] * counts[a] for a in KD.keys()) / L

    for aa in KD.keys():
        feats[f"{prefix}_comp_{aa}"] = counts[aa] / L

    return feats


In [8]:
def engineer_sequence_features(df, seq_cols=("VH", "VL")):
    rows = []
    for _, row in df.iterrows():
        feats = {}
        for col in seq_cols:
            feats.update(seq_features(row.get(col, None), col))
        rows.append(feats)
    feat_df = pd.DataFrame(rows, index=df.index)
    return feat_df

In [9]:
def build_model_ready_from_merged(merged_df, target_col="Slope for Accelerated Stability", seq_cols=("VH", "VL"), include_assays=False, impute_strategy="median"):
    if target_col in merged_df.columns:
        df = merged_df.dropna(subset=[target_col]).copy()
    else:
        df = merged_df.copy()

    seq_feats = engineer_sequence_features(df, seq_cols=seq_cols)

    if include_assays:
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if target_col in num_cols:
            num_cols.remove(target_col)
        assay_feats = df[num_cols].copy()
    else:
        assay_feats = pd.DataFrame(index=df.index)

    X_raw = pd.concat([seq_feats, assay_feats], axis=1)
    feature_columns = X_raw.columns.tolist()
    y = df[target_col].values if target_col in df.columns else None

    imputer = SimpleImputer(strategy=impute_strategy)
    scaler = StandardScaler()
    X_imputed = imputer.fit_transform(X_raw)
    X_scaled = scaler.fit_transform(X_imputed)
    X = pd.DataFrame(X_scaled, index=df.index, columns=feature_columns)

    return {
        "X" : X,
        "X_pca_input": X.copy(),
        "y": y,
        "feature_columns": feature_columns,
        "supervised_df": df,
        "imputer": imputer,
        "scaler": scaler
    }


In [10]:
directory = os.getcwd()
data_files = scan_directory(directory, file_extension=".xlsx")
data_files

['data/pnas.1616408114.sd02.xlsx',
 'data/pnas.1616408114.sd03.xlsx',
 'data/pnas.1616408114.sd01.xlsx']

In [11]:
dfs = read_files(data_files)
dfs

[            Name                                                 VH  \
 0     abituzumab  QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...   
 1      abrilumab  QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...   
 2     adalimumab  EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...   
 3    alemtuzumab  QVQLQESGPGLVRPSQTLSLTCTVSGFTFTDFYMNWVRQPPGRGLE...   
 4     alirocumab  EVQLVESGGGLVQPGGSLRLSCAASGFTFNNYAMNWVRQAPGKGLD...   
 ..           ...                                                ...   
 132  vedolizumab  QVQLVQSGAEVKKPGASVKVSCKGSGYTFTSYWMHWVRQAPGQRLE...   
 133   veltuzumab  QVQLQQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVKQAPGQGLE...   
 134  visilizumab  QVQLVQSGAEVKKPGASVKVSCKASGYTFISYTMHWVRQAPGQGLE...   
 135  zalutumumab  QVQLVESGGGVVQPGRSLRLSCAASGFTFSTYGMHWVRQAPGKGLE...   
 136  zanolimumab  QVQLQQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQPPGKGLE...   
 
                                                     VL LC Class     Source  \
 0    DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...

In [12]:
merged_df = create_clean_df(dfs)
merged_df

,Name,VH,VL,LC Class,Source,Source Detaileda,Disclaimers and Known Issues,Notes_x,HEK Titer (mg/L),Fab Tm by DSF (°C),...,CSI-BLI Delta Response (nm),ELISA,BVP ELISA,Light chain class,Type,Original mAb Isotype or Format,Clinical Status,Phagec,Year Name Proposed,Notes_y
0,abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,kappa,WHO-INN,PL109,NaN,aPL and RL refer to WHO-INN publications for p...,89.555458,75.5,...,0.00,1.137375,2.720799,kappa,ZU,IgG2,Phase 2,No,2013,a Made with human isotype
1,abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,kappa,WHO-INN,PL111,NaN,NaN,100.223196,71.0,...,-0.02,1.124624,1.818303,kappa,HU,IgG2,Phase 2,No,2014,"b Mixed chimeric, humanized LC and HC"
2,adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,kappa,PDB,4NYL,NaN,NaN,134.928638,71.0,...,-0.01,1.075515,1.488186,kappa,HU,IgG1,Approved,Yes,1999,c Most as those molecules labeled Yes in the p...
3,alemtuzumab,QVQLQESGPGLVRPSQTLSLTCTVSGFTFTDFYMNWVRQPPGRGLE...,DIQMTQSPSSLSASVGDRVTITCKASQNIDKYLNWYQQKPGKAPKL...,kappa,PDB,1BEY,NaN,NaN,144.653543,74.5,...,-0.02,1.161491,1.464226,kappa,ZU,IgG1,Approved,No,2000,NaN
4,alirocumab,EVQLVESGGGLVQPGGSLRLSCAASGFTFNNYAMNWVRQAPGKGLD...,DIVMTQSPDSLAVSLGERATINCKSSQSVLYRSNNRNFLGWYQQKP...,kappa,WHO-INN,PL107,NaN,NaN,69.232345,71.5,...,-0.01,1.196224,2.179700,kappa,HU,IgG1,Approved,No,2012,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,vedolizumab,QVQLVQSGAEVKKPGASVKVSCKGSGYTFTSYWMHWVRQAPGQRLE...,DVVMTQSPLSLPVTPGEPASISCRSSQSLAKSYGNTYLSWYLQKPG...,kappa,WHO-INN,PL100,NaN,NaN,221.762037,80.5,...,-0.02,1.152671,1.584027,kappa,ZU,IgG1AA-mut,Approved,No,2008,NaN
133,veltuzumab,QVQLQQSGAEVKKPGSSVKVSCKASGYTFTSYNMHWVKQAPGQGLE...,DIQLTQSPSSLSASVGDRVTMTCRASSSVSYIHWFQQKPGKAPKPW...,kappa,WHO-INN,PL98,NaN,NaN,224.953517,70.0,...,-0.02,0.888809,1.211236,kappa,ZU,IgG1,Phase 2,No,2007,NaN
134,visilizumab,QVQLVQSGAEVKKPGASVKVSCKASGYTFISYTMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCSASSSVSYMNWYQQKPGKAPKRL...,kappa,US Patent,US7381803,NaN,NaN,242.006377,71.0,...,0.01,1.880772,4.799334,kappa,ZU,IgG2,Phase 3,No,2000,NaN
135,zalutumumab,QVQLVESGGGVVQPGRSLRLSCAASGFTFSTYGMHWVRQAPGKGLE...,AIQLTQSPSSLSASVGDRVTITCRASQDISSALVWYQQKPGKAPKL...,kappa,US Patent,US8586041,NaN,NaN,200.506690,72.5,...,-0.03,1.284363,2.896506,kappa,HU,IgG1,Phase 3,No,2005,NaN


In [13]:
model_ready_df = build_model_ready_from_merged(merged_df, target_col="Slope for Accelerated Stability", seq_cols=("VH", "VL"), include_assays=False, impute_strategy="median")

In [14]:
# # Mengayo's Original Code

# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.neural_network import MLPRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import root_mean_squared_error, r2_score

# # ----------  IMPORT DATA PREP FUNCTIONS  ----------
# # from data_prep import (      # save their code as data_cleaning.py
# #     scan_directory, read_files, create_clean_df,
# #     build_model_ready_from_merged
# # )

# # ----------  PLOTS  ----------
# def plot_pred_vs_obs(y_train, y_train_pred, y_test, y_test_pred, title):
#     plt.figure(figsize=(6,6))
#     plt.scatter(y_train, y_train_pred, c="blue", label="Train", alpha=0.6)
#     plt.scatter(y_test,  y_test_pred,  c="red",  label="Test",  alpha=0.6)
#     minv, maxv = min(y_train.min(), y_test.min()), max(y_train.max(), y_test.max())
#     plt.plot([minv, maxv], [minv, maxv], "k--")
#     rmse  = root_mean_squared_error(y_test, y_test_pred)
#     r2    = r2_score(y_test, y_test_pred)
#     plt.title(f"{title}\nTest RMSE={rmse:.2f}, R²={r2:.2f}")
#     plt.xlabel("Observed")
#     plt.ylabel("Predicted")
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

# def plot_residuals(y_train, y_train_pred, y_test, y_test_pred, title):
#     plt.figure(figsize=(7,4))
#     plt.scatter(y_train_pred, y_train - y_train_pred,
#                 c="blue", label="Train", alpha=0.6)
#     plt.scatter(y_test_pred,  y_test - y_test_pred,
#                 c="red",  label="Test",  alpha=0.6)
#     plt.axhline(0, color="black", linestyle="--")
#     plt.xlabel("Predicted")
#     plt.ylabel("Residual (Obs – Pred)")
#     plt.title(f"{title} Residuals")
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

# # ----------  MODEL TRAIN / EVAL  ----------
# def train_and_eval(X, y, model_name):
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.2, random_state=41
#     )


#     # GridSearchCV will conduct an additional sub split within the already split data
#     if model_name == "rf":
#         model = GridSearchCV(
#             RandomForestRegressor(random_state=42),
#             param_grid={"n_estimators":[10, 50, 200],
#                         "max_depth":[None,20,40]},
#             n_jobs=-1,
#             scoring="neg_root_mean_squared_error"
#         )
#     elif model_name == "gb":
#         model = GridSearchCV(
#             GradientBoostingRegressor(random_state=42),
#             param_grid={"n_estimators":[10, 50, 200],
#                         "learning_rate":[0.05,0.1],
#                         "max_depth":[1,3,5]},
#             n_jobs=-1,
#             scoring="neg_root_mean_squared_error"
#         )
#     elif model_name == "mlp":
#         model = GridSearchCV(
#             MLPRegressor(max_iter=500, random_state=42),
#             param_grid={"hidden_layer_sizes":[(64,32), (128,64),(256,128)],
#                         "alpha":[1e-2, 0.1]},
#             n_jobs=-1,
#             scoring="neg_root_mean_squared_error"
#         )
#     elif model_name == "svr":
#         # RBF kernel with a small grid of C and gamma
#         model = GridSearchCV(
#             SVR(kernel="rbf"),
#             param_grid={"C":[1.0,10.0,100.0],
#                         "gamma":["scale","auto",0.1]},
#             n_jobs=-1,
#             scoring="neg_root_mean_squared_error"
#         )
#     else:
#         raise ValueError("Choose model_name from: rf, gb, mlp, svr")

#     model.fit(X_train, y_train)
#     print(f"\n=== {model_name.upper()} ===")
#     print("Best Params:", model.best_params_)

#     y_train_pred = model.predict(X_train)
#     y_test_pred  = model.predict(X_test)

#     print(f"Train RMSE: {root_mean_squared_error(y_train, y_train_pred):.3f}, "
#           f"Train R²: {r2_score(y_train, y_train_pred):.3f}")
#     print(f"Test  RMSE: {root_mean_squared_error(y_test, y_test_pred):.3f}, "
#           f"Test  R²: {r2_score(y_test, y_test_pred):.3f}")

#     # plot_pred_vs_obs(y_train, y_train_pred, y_test, y_test_pred, model_name.upper())
#     # plot_residuals(y_train, y_train_pred, y_test, y_test_pred, model_name.upper())

# # To be compiled into reg_model() function 
# X, y = model_ready_df["X"].values, model_ready_df["y"]

# # Train/evaluate each requested model
# for m in ["rf","gb","mlp","svr"]:
#     train_and_eval(X, y, m)

In [15]:
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

rng = 42
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=rng)

def train_and_eval(X, y, model_name):
    if model_name == "rf":
        est = RandomForestRegressor(random_state=rng, n_jobs=-1)
        grid = {
            "n_estimators": [100, 200],
            "max_depth": [8, 12, 16],
            "min_samples_leaf": [2, 4, 8]
        }

    elif model_name == "gb":
        est = GradientBoostingRegressor(random_state=rng)
        grid = {
            "n_estimators": [100, 200],
            "learning_rate": [0.05, 0.1],
            "max_depth": [2, 3],
            "min_samples_leaf": [2, 4, 8],
            "subsample": [0.7, 1.0]
        }
    
    elif model_name == "mlp":
        est = MLPRegressor(
            hidden_layer_sizes=(128, 64),
            max_iter=1000,
            early_stopping=True,
            validation_fraction=0.15,
            random_state=rng
        )
        grid = {"alpha": [1e-3, 1e-2]}

    elif model_name == "svr":
        est = SVR(kernel="rbf")
        grid = {
            "C": [1.0, 10.0, 50.0],
            "gamma": ["scale", 0.001, 0.1]
        }
    
    else:
        raise ValueError("rf | bg | mlp | svr")
    
    gs = GridSearchCV(
        est, grid, cv=cv, n_jobs=-1,
        scoring="neg_root_mean_squared_error", refit=True
    )

    gs.fit(X, y)

    y_pred = gs.predict(X)
    rmse = root_mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)

    print(f"\n=== {model_name.upper()} ===")
    print("Best Params:", gs.best_params_)
    print(f"CV best RMSE: {-gs.best_score_:.3f}")
    print(f"In-sample RMSE: {rmse:.3f} | R-squared: {r2:.3f} | MAE: {mae:.3f}")

    return gs

In [16]:
X = model_ready_df["X"]
y = model_ready_df["y"]

models = {}
for m in ["rf", "gb", "mlp", "svr"]:
    models[m] = train_and_eval(X, y, m)


=== RF ===
Best Params: {'max_depth': 8, 'min_samples_leaf': 8, 'n_estimators': 100}
CV best RMSE: 0.126
In-sample RMSE: 0.101 | R-squared: 0.259 | MAE: 0.038

=== GB ===
Best Params: {'learning_rate': 0.05, 'max_depth': 2, 'min_samples_leaf': 8, 'n_estimators': 100, 'subsample': 0.7}
CV best RMSE: 0.129
In-sample RMSE: 0.081 | R-squared: 0.514 | MAE: 0.036

=== MLP ===
Best Params: {'alpha': 0.01}
CV best RMSE: 0.335
In-sample RMSE: 0.152 | R-squared: -0.687 | MAE: 0.116

=== SVR ===
Best Params: {'C': 1.0, 'gamma': 0.001}
CV best RMSE: 0.135
In-sample RMSE: 0.114 | R-squared: 0.051 | MAE: 0.063
